In [ ]:
from boring_model.model import BoringDataModule
import torch
import lightning as L
from torch import nn

In [ ]:
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(1, 3)
        self.l2 = nn.Linear(3, 1)

    def forward(self, x):
        return self.l2(self.l1(x))

In [43]:
class BoringModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.lr = 1e-3
        self.save_hyperparameters()
        self.criterion = torch.nn.MSELoss()
        self.model = LinearModel()

        self.automatic_optimization = False

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters())
        return optimizer

    def training_step(self, data):
        optimizer = self.optimizers()
        optimizer.zero_grad()

        x, y = data
        o = self(x)
        loss = self.criterion(o, y)
        self.log("loss", loss)

        self.manual_backward(loss)
        optimizer.step()
        return loss
    
    def validation_step(self, data, *args, **kwargs):
        x, y = data
        o = self(x)
        loss = self.criterion(o, y)
        self.log("val_loss", loss)
        return loss
    
    def test_step(self, data, *args, **kwargs):
        x, y = data
        o = self(x)
        loss = self.criterion(o, y)
        self.log("test_loss", loss)
        return loss

In [49]:
model = BoringModel()
dm = BoringDataModule()
trainer = L.Trainer(max_epochs=20)

trainer.fit(model, datamodule=dm)
trainer.validate(model, datamodule=dm)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type        | Params
------------------------------------------
0 | criterion | MSELoss     | 0     
1 | model     | LinearModel | 10    
------------------------------------------
10        Trainable params
0         Non-trainable params
10        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    1.3142186403274536     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 1.3142186403274536}]

In [48]:
model(torch.tensor([[194342.,]]))

tensor([[322986.9062]], grad_fn=<AddmmBackward0>)

In [47]:
194342*2

388684